<h1 style="text-align : center"> Docker </h1>

# Table of Contents
* [How it works on Windows / OSX](#How-it-works-on-Windows-/-OSX)
* [Troubleshooting on windows](#Troubleshooting-on-windows)
* [Images and Containers](#Images-and-Containers)
* [Containers vs. VM](#Containers-vs.-VM)
* [LXC (Linux Containers)](#LXC-%28Linux-Containers%29)
* [Dockerfile](#Dockerfile)
* [Common operations](#Common-operations)
	* [Attach / Detach from a container](#Attach-/-Detach-from-a-container)
* [Git-like capabilities](#Git-like-capabilities)
	* [Commit the state of a container](#Commit-the-state-of-a-container)
	* [Revert to a previous state](#Revert-to-a-previous-state)


# How it works on Windows / OSX

Docker needs a linux kernel in order to wor properly, so if our OS is different we need to create a vm with a linux kernel and install docker on it.

Basically, the way docker works is that the **docker ...** commands are passed to a docker service (running on port 2375).

- **Flow :**
    1. The default virtual machine is created in VirtualBox
    2. the starter (kinamatic / "Docker Quickstart Terminal" / cmd) of the machine waits for the machine to boot, gets it's IP address and sets the default vm ip:port as the service address in few environment variables
    3. from this point all **docker ...** commands are send to the linux host and everything works almost as native linux

# Troubleshooting on windows

I have encountered some problem during the installation of docker on my windows machine:
* bash.exe not found
    * just search manually the path to bash.exe (it should be in the Git directory)    
* Docker Quickstart Terminal doesn't work the first time you launch it
    * launch the Docker Quickstart Terminal multiple times
* No connection could be made because the target machine actively refused it
    * Set the env variables manually with:
    ```bash
    "C:\Program Files\Docker Toolbox\docker-machine" env default --shell=cmd
    ```
    **NB :** default is the name of the vm retived with:
    ```bash
    "C:\Program Files\Docker Toolbox\docker-machine" ls
    ```
* Open C:\Users\X\.docker\machine\machines\default\server.pem: The system cannot find the file specified.
    * regenerate the vm certificates with:
    ```bash
    docker-machine regenerate-cers <VM_NAME>
    ```
    and restart the Docker Quickstart Terminal

# Images and Containers

- **Containers :** Stripped-to-basic linux operating system
- **Images :** Softaware you load into a container

**ES :** ```docker run hello-world``` : load the image "hello-wold" in the container and ran it

Every image is **layered**. A layer is a sort pf snapshot ofthe state of the machine.

# Containers vs. VM

<img  src="GUL54KKUVLLU1FQJ4GM7UHPJUOUT9WJ1.png"/>
- **VMs :** have a full operating system with iots own memory managements, device drivers, etc...
- **Containers :** Share the hos OS and its memory management, device drivers, etc..

Containers create an isolated environment for your image and each of them has its own runtime environment and its own network stack and process space.
each container **has access only to the ports and files explicitly exposed by other container**.

# LXC (Linux Containers)

Docker is built on the top of **LXC**.
LXC refers to capabilities of the linux kernel which allow sandboxing processes from one another and controlling the resource allocations.
Docker offers additional capabilities respect to LXC such as versioning and sharing of the image.

# Dockerfile

A Dockerfile is the starting point of your docker image that has to be loaded.
Here you can specify various action that will be executed when you **build** the image.

- **Keywords :**
    - **FROM** : specify the which image is the base for your image
    - **RUN** : the command specified is executed during the **build phase**
    - **CMD** : the command specified is executed **after the image is instatiated**
    - **EXPOSE :** specify which ports expose to the outside world
    - **VOLUME :** mount the specified directory of the host machine in the container

# Common operations

## Attach / Detach from a container

In order to be able to attach and then detach from a container leaving it in a running state, we have to execute this list of commands:

1. **```docker run -t -i <IMAGE_NAME:TAG>```** : create a container in a stopped state (-i flag leaves STDIN open even f nobody is attached to it)

2. **```docker start <CONTAINER_ID>```** : start the specified container

3. **```docker attach <CONTAINER_ID>```** : attach to the specified container (get a shell on it)

4. <kbd>CTRL</kbd> + <kbd>Z</kbd> + <kbd>CTRL</kbd> + <kbd>Q</kbd> : detatch from the container leaving it in a running state

# Git-like capabilities

## Commit the state of a container

The flesystem of a container is not persistent. Only when we install a program the changes become persistent. This happen because every time a program is intstalled a new image layer is created.

If we want to create/modify a file and make this change permanent we have to **detatch** from the container that we want to update and then  **commit** the state of the container with:

**```docker commit [-m <MESSAGE>] <CONTANER_ID> <IMAGE_NAME:TAG>```**

<img  src="TC8FOJC64FS0PT9PWPHP9QXR74AAPAFH.png"/>

We can have two cases:
- If the IMAGE_NAME:TAG already exist the commit will create an additional layer in the specified image
- If the IMAGE_NAME:TAG oesn't exist, docker will create another image with the specified IMAGE_NAME:TAG and with the same history of the one running in the container plus the new layer

## Revert to a previous state

We can see all the commit history (from the current one to the older one) of a container with:

- **```docker history <IMAGE_NAME || IMAGE_ID>```**

<img  src="PQ9LUBMFN2WG8B27HUGWXXYU1BFKDD82.png"/>

In order to rollback to a previous state we have to **tag** the image to the IMAGE_ID of the chosen state with:

- **```docker tag <IMAGE_ID> <IMAGE_NAME:TAG>```**

<img  src="Q2AB80KVHHVI73XDKIBMJE8DN1F3U0OQ.png"/>

As we can see is not a **real** rollback, Docker just creates another image with tagged to the tag chosen before (kali:reverted at 46a5181c22a1)

<img  src="NB2FFRW6JLS1A4BF1JXMH5LSLVUV6V0B.png"/>

Here the status with the comments "my second commit" is gone and teh filesystem has been restored to the state it was on the tag "my first commit"